In [1]:
import os

In [2]:
%pwd

'c:\\Users\\vishpand\\Desktop\\projects\\e2e-mlops-project\\notebooks'

In [3]:
os.chdir('..')

In [4]:
%pwd

'c:\\Users\\vishpand\\Desktop\\projects\\e2e-mlops-project'

In [43]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [44]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories, save_json

In [45]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
            ) -> None:
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name= config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/im-vishal/e2e-mlops-project.mlflow"
        )

        return model_evaluation_config

In [46]:
from pathlib import Path
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import pandas as pd
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [47]:
!pip install python-dotenv

In [48]:
import os
from dotenv import load_dotenv

In [49]:
import os
import dagshub

# Set environment variables for DagsHub credentials if not already set
os.environ['MLFlow_TRACKING_URI'] = "https://dagshub.com/im-vishal/e2e-mlops-project.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = 'im-vishal'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '2e6d8d488ffd0ad8d3644ab97470d94ebce15e29'



In [50]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig) -> None:
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_artifact_uri()).scheme
        # mlflow.set_tracking_uri(self.config.mlflow_uri)

        try:
            # Check if there's an active run and end it if necessary
            if mlflow.active_run():
                mlflow.end_run()

                
            with mlflow.start_run():
                predicted_qualities = model.predict(test_x)

                (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

                # Saving metrics as local
                scores = {"rmse": rmse, "mae": mae, "r2": r2}
                save_json(path=Path(self.config.metric_file_name), data=scores)

                mlflow.log_params(self.config.all_params)
                
                mlflow.log_metric("rmse", rmse)
                mlflow.log_metric("r2", r2)
                mlflow.log_metric("mae", mae)


                # Model registry does not work with file store
                if tracking_url_type_store != "file":
                    mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
                else:
                    mlflow.sklearn.log_model(model, "model")
        finally:
            # End the MLflow run explicitly if not using `with` statement
            if mlflow.active_run():
                mlflow.end_run()

In [51]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-07-05 11:20:59,649: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-05 11:20:59,652: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-05 11:20:59,657: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-05 11:20:59,660: INFO: common: created directory at: artifacts]
[2024-07-05 11:20:59,662: INFO: common: created directory at: artifacts/model_evaluation]
[2024-07-05 11:21:01,431: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2024/07/05 11:21:12 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticNetModel, version 2
Created version '2' of model 'ElasticNetModel'.


In [52]:
import os
from dotenv import load_dotenv

In [53]:
load_dotenv()

True